# Lab 5: Neural Network Classification with scikit-learn

---
## 1. Notebook Overview

### 1.1 Objective
- Re-use the most frequent words (optional: per class) you found for
your Naive Bayes classifier last week.

- Construct binary vectors for your whole dataset. Each dimension states
whether the word is part of the sample or not.

- Create a small neural network using scikit-learn: https://scikit-learn.org/
stable/modules/neural_networks_supervised.html. Start with three
hidden layers of 128/64/128 neurons. Consider what your input and
output layers should look like.

- Train your network on your training set and test it on your test set.
Calculate evaluation measures and compare with your previous
classifier.

- Optional: Experiment with different network sizes.

### 1.2 Prerequisites
This notebook assumes you have already executed:
- **Lab 2**: Data preprocessing → `../Data/tweets_preprocessed_train.parquet`, `../Data/tweets_preprocessed_test.parquet`, `../Data/tweets_preprocessed_validation.parquet`
- **Lab 3**: Language modeling
- **Lab 4**: Feature extraction → `../Data/top_1000_vocabulary.json`

### 1.3 Architecture
We implement a neural network with:
- **Input layer**: 1000 features (Top 1000 vocabulary from Lab 4)
- **Hidden layers**: 128 → 64 → 128 neurons (as specified)
- **Output layer**: 19 binary classifiers (one per topic class, using OneVsRestClassifier)

### 1.4 Neural Network Fundamentals (From Lecture)
- A single neuron computes: ŷ = g(w₀ + Σ xᵢwᵢ) where g is a non-linear activation function
- **Activation functions are critical** - they introduce non-linearities that make multi-layer networks powerful (universal approximators)
- Common activations: ReLU (g(z) = max(0,z)), Sigmoid, Tanh
- For multi-class (single-label): use **Softmax** to convert outputs to probabilities
- For multi-label (our case): use **Sigmoid** per class via OneVsRestClassifier
- **Loss function for classification**: Cross-entropy loss
- Weights should NOT be initialized to all zeros (breaks symmetry)

---
## 2. Task 1: Establish Context

### 2.1 Review Preprocessing from Lab 2
In Lab 2, we preprocessed tweets with the following pipeline:
- Remove RT indicators, URLs, usernames, and mentions
- Convert emojis to text descriptions
- Extract hashtag text and segment CamelCase words
- Normalize whitespace and lowercase
- Tokenize with SpaCy and filter/lemmatize tokens

The output is stored in parquet files with columns: `text`, `label_name`, `label`

Two approaches for label handling are supported:
- Parse `label_name` (string list format) into Python lists
- Use `label` column directly (pre-computed binary vectors)

In [ ]:
# Import required libraries
import json
import ast
from typing import List

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    precision_score, 
    recall_score, 
    hamming_loss,
    classification_report
)
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier

# Constants
TRAIN_DATA_PATH = "../Data/tweets_preprocessed_train.parquet"
TEST_DATA_PATH = "../Data/tweets_preprocessed_test.parquet"
VALIDATION_DATA_PATH = "../Data/tweets_preprocessed_validation.parquet"
VOCABULARY_PATH = "../Data/top_1000_vocabulary.json"
RANDOM_STATE = 42

print("✓ Libraries imported successfully")

### 2.2 Load and Verify Vocabulary from Lab 4

In [ ]:
# Load the top 1000 vocabulary from Lab 4
with open(VOCABULARY_PATH, 'r', encoding='utf-8') as f:
    vocab_data = json.load(f)

VOCABULARY = vocab_data['tokens']
vocab_set = set(VOCABULARY)

print(f"✓ Loaded vocabulary from: {VOCABULARY_PATH}")
print(f"✓ Description: {vocab_data['description']}")
print(f"✓ Vocabulary size: {len(VOCABULARY)}")
print(f"✓ First 20 tokens: {VOCABULARY[:20]}")
print(f"✓ Last 10 tokens: {VOCABULARY[-10:]}")

### 2.3 Load Preprocessed Datasets

In [ ]:
def parse_labels(value) -> List[str]:
    """Parse label_name column into consistent Python lists."""
    if isinstance(value, (list, np.ndarray)):
        return [str(v) for v in value]
    if isinstance(value, tuple):
        return [str(v) for v in value]
    if isinstance(value, str):
        value = value.strip()
        if value.startswith('[') and value.endswith(']'):
            # Remove brackets
            inner = value[1:-1].strip()
            if not inner:
                return []
            # Remove quotes and split by whitespace (handles both formats)
            inner = inner.replace("'", "").replace('"', '')
            labels = [l.strip() for l in inner.split() if l.strip()]
            return labels
        try:
            parsed = ast.literal_eval(value)
            if isinstance(parsed, (list, tuple)):
                return [str(v) for v in parsed]
        except (ValueError, SyntaxError):
            pass
        return [value] if value else []
    return [str(value)] if value else []

def parse_binary_label(value) -> np.ndarray:
    """Parse binary label array from string representation."""
    if isinstance(value, np.ndarray):
        return value
    if isinstance(value, str):
        # Parse "[0 0 1 0 ...]" format
        inner = value.strip()[1:-1]
        return np.array([int(x) for x in inner.split()])
    return np.array(value)

def load_dataset(path: str) -> pd.DataFrame:
    """Load tweets from parquet and normalize the label columns."""
    df = pd.read_parquet(path)
    df = df.copy()
    df["labels"] = df["label_name"].apply(parse_labels)
    df["label_binary"] = df["label"].apply(parse_binary_label)
    return df

# Load all datasets
df_train = load_dataset(TRAIN_DATA_PATH)
df_test = load_dataset(TEST_DATA_PATH)
df_validation = load_dataset(VALIDATION_DATA_PATH)

print(f"✓ Training set: {len(df_train):,} samples")
print(f"✓ Test set: {len(df_test):,} samples")
print(f"✓ Validation set: {len(df_validation):,} samples")
print(f"\nSample preprocessed text:")
print(f"  {df_train['text'].iloc[0][:80]}...")
print(f"  Labels: {df_train['labels'].iloc[0]}")

---
## 3. Task 2: Implementation Plan

### 3.1 Binary Feature Vector Construction
For each sample, we create a binary vector of size 1000 (vocabulary size):
- For each word in the vocabulary, set dimension to 1 if word is present in sample, 0 otherwise
- This is a Bag-of-Words style encoding (word order is lost)

### 3.2 MLPClassifier Configuration
- **hidden_layer_sizes**: (128, 64, 128) - three hidden layers as specified
- **activation**: 'relu' - ReLU activation (most commonly used)
- **solver**: 'adam' - Adam optimizer (handles mini-batch gradient descent)
- **max_iter**: 300 - sufficient iterations for convergence
- **random_state**: 42 - for reproducibility
- **early_stopping**: Disabled (some classes have few samples in multi-label setting)

### 3.3 Evaluation Metrics
For comparison with Naive Bayes from Lab 4:
- Subset Accuracy (exact match)
- Hamming Loss
- Micro/Macro F1-Score
- Micro Precision and Recall

---
## 4. Task 3: Implementation

### 4.1 Feature Engineering: Binary Vector Construction

In [ ]:
def create_binary_features(texts: pd.Series, vocabulary: List[str]) -> np.ndarray:
    """
    Create binary feature vectors for text samples.
    
    Each dimension represents whether a word from the vocabulary
    is present (1) or absent (0) in the sample.
    
    Parameters:
    -----------
    texts : pd.Series
        Series of preprocessed text strings (whitespace-tokenized)
    vocabulary : List[str]
        List of vocabulary words (top 1000 from Lab 4)
    
    Returns:
    --------
    np.ndarray
        Binary feature matrix of shape (n_samples, vocab_size)
    """
    vocab_set = set(vocabulary)
    vocab_to_idx = {word: idx for idx, word in enumerate(vocabulary)}
    
    n_samples = len(texts)
    n_features = len(vocabulary)
    
    # Initialize feature matrix with zeros
    features = np.zeros((n_samples, n_features), dtype=np.int8)
    
    # Fill in binary features
    for i, text in enumerate(texts):
        if isinstance(text, str):
            words = set(text.split())
            for word in words:
                if word in vocab_to_idx:
                    features[i, vocab_to_idx[word]] = 1
    
    return features

# Create binary feature vectors for all datasets
print("Creating binary feature vectors...")
X_train = create_binary_features(df_train['text'], VOCABULARY)
X_test = create_binary_features(df_test['text'], VOCABULARY)
X_validation = create_binary_features(df_validation['text'], VOCABULARY)

print(f"\n✓ Feature matrix shapes:")
print(f"  X_train: {X_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  X_validation: {X_validation.shape}")

# Show sample feature statistics
print(f"\nFeature statistics (training set):")
print(f"  Average features per sample: {X_train.sum(axis=1).mean():.2f}")
print(f"  Max features in a sample: {X_train.sum(axis=1).max()}")
print(f"  Min features in a sample: {X_train.sum(axis=1).min()}")

### 4.2 Label Encoding (Multi-Label Binarization)

In [ ]:
# Define the 19 topic classes (from the original dataset)
TOPIC_CLASSES = [
    'arts_&_culture', 'business_&_entrepreneurs', 'celebrity_&_pop_culture',
    'diaries_&_daily_life', 'family', 'fashion_&_style', 'film_tv_&_video',
    'fitness_&_health', 'food_&_dining', 'gaming', 'learning_&_educational',
    'music', 'news_&_social_concern', 'other_hobbies', 'relationships',
    'science_&_technology', 'sports', 'travel_&_adventure', 'youth_&_student_life'
]

# Use the pre-parsed binary labels directly from the label column
y_train = np.vstack(df_train['label_binary'].values)
y_test = np.vstack(df_test['label_binary'].values)
y_validation = np.vstack(df_validation['label_binary'].values)

# Create MultiLabelBinarizer for inverse_transform (label names)
mlb = MultiLabelBinarizer(classes=TOPIC_CLASSES)
mlb.fit([TOPIC_CLASSES])  # Fit with all classes

print(f"✓ Number of classes: {len(TOPIC_CLASSES)}")
print(f"✓ Classes: {TOPIC_CLASSES}")
print(f"\n✓ Label matrix shapes:")
print(f"  y_train: {y_train.shape}")
print(f"  y_test: {y_test.shape}")
print(f"  y_validation: {y_validation.shape}")

# Verify label distribution
print(f"\n✓ Label distribution (training set):")
print(f"  Average labels per sample: {y_train.sum(axis=1).mean():.2f}")
print(f"  Samples per class: {y_train.sum(axis=0)}")

### 4.3 Neural Network Model Training

In [ ]:
# Create MLPClassifier with specified architecture
# Using OneVsRestClassifier for multi-label classification
# Note: early_stopping is disabled because some classes have very few samples
# which causes issues with the validation split in OneVsRest multi-label setting
mlp_base = MLPClassifier(
    hidden_layer_sizes=(128, 64, 128),  # Three hidden layers as specified
    activation='relu',                   # ReLU activation function
    solver='adam',                       # Adam optimizer (mini-batch gradient descent)
    max_iter=300,                        # Maximum iterations
    random_state=RANDOM_STATE,           # For reproducibility
    early_stopping=False,                # Disabled for multi-label compatibility
    verbose=True                         # Show training progress
)

# Wrap with OneVsRestClassifier for multi-label support
mlp_clf = OneVsRestClassifier(mlp_base, n_jobs=-1)

print("="*60)
print("NEURAL NETWORK ARCHITECTURE")
print("="*60)
print(f"Input layer:  {X_train.shape[1]} neurons (vocabulary size)")
print(f"Hidden layer 1: 128 neurons (ReLU activation)")
print(f"Hidden layer 2: 64 neurons (ReLU activation)")
print(f"Hidden layer 3: 128 neurons (ReLU activation)")
print(f"Output layer: {len(TOPIC_CLASSES)} neurons (19 topic classes)")
print("="*60)

print("\nTraining Neural Network...")
mlp_clf.fit(X_train, y_train)
print("\n✓ Neural Network training complete!")

### 4.4 Neural Network Evaluation

In [ ]:
# Make predictions
y_pred_nn = mlp_clf.predict(X_test)

# Calculate metrics
nn_metrics = {
    'Subset Accuracy': accuracy_score(y_test, y_pred_nn),
    'Hamming Loss': hamming_loss(y_test, y_pred_nn),
    'Micro F1': f1_score(y_test, y_pred_nn, average='micro', zero_division=0),
    'Macro F1': f1_score(y_test, y_pred_nn, average='macro', zero_division=0),
    'Micro Precision': precision_score(y_test, y_pred_nn, average='micro', zero_division=0),
    'Micro Recall': recall_score(y_test, y_pred_nn, average='micro', zero_division=0)
}

print("="*60)
print("NEURAL NETWORK EVALUATION (Test Set)")
print("="*60)
for metric, value in nn_metrics.items():
    print(f"{metric:<20}: {value:.4f}")

In [ ]:
# Show sample predictions
y_pred_labels = mlb.inverse_transform(y_pred_nn)
y_true_labels = mlb.inverse_transform(y_test)

print("\nSample Neural Network Predictions:")
print("-" * 60)
for i in range(5):
    text = df_test['text'].iloc[i][:60]
    true = y_true_labels[i] if y_true_labels[i] else ('none',)
    pred = y_pred_labels[i] if y_pred_labels[i] else ('none',)
    match = "✓" if set(true) == set(pred) else "✗"
    print(f"\n{match} Sample {i+1}:")
    print(f"   Text: {text}...")
    print(f"   True: {true}")
    print(f"   Pred: {pred}")

### 4.5 Naive Bayes Classifier (for Comparison)

In [ ]:
# Train Naive Bayes classifier with same features
nb_clf = OneVsRestClassifier(MultinomialNB(alpha=1.0))
nb_clf.fit(X_train, y_train)

# Make predictions
y_pred_nb = nb_clf.predict(X_test)

# Calculate metrics
nb_metrics = {
    'Subset Accuracy': accuracy_score(y_test, y_pred_nb),
    'Hamming Loss': hamming_loss(y_test, y_pred_nb),
    'Micro F1': f1_score(y_test, y_pred_nb, average='micro', zero_division=0),
    'Macro F1': f1_score(y_test, y_pred_nb, average='macro', zero_division=0),
    'Micro Precision': precision_score(y_test, y_pred_nb, average='micro', zero_division=0),
    'Micro Recall': recall_score(y_test, y_pred_nb, average='micro', zero_division=0)
}

print("="*60)
print("NAIVE BAYES EVALUATION (Test Set)")
print("="*60)
for metric, value in nb_metrics.items():
    print(f"{metric:<20}: {value:.4f}")

### 4.6 Model Comparison: Neural Network vs Naive Bayes

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Metric': list(nn_metrics.keys()),
    'Neural Network': list(nn_metrics.values()),
    'Naive Bayes': list(nb_metrics.values())
})

# Calculate improvement
comparison_df['Difference'] = comparison_df['Neural Network'] - comparison_df['Naive Bayes']
comparison_df['Better Model'] = comparison_df.apply(
    lambda row: 'Neural Network' if (row['Difference'] > 0 and row['Metric'] != 'Hamming Loss') 
                or (row['Difference'] < 0 and row['Metric'] == 'Hamming Loss')
                else 'Naive Bayes' if row['Difference'] != 0 else 'Tie',
    axis=1
)

print("="*80)
print("MODEL COMPARISON: Neural Network vs Naive Bayes")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)
print("\nNote: For Hamming Loss, lower is better. For all other metrics, higher is better.")

---
## 5. Optional: Experiment with Different Network Sizes

In [ ]:
# Define different architectures to test
architectures = {
    'Small (64-32-64)': (64, 32, 64),
    'Medium (128-64-128)': (128, 64, 128),  # Original
    'Large (256-128-256)': (256, 128, 256),
    'Deep (128-128-64-64-128-128)': (128, 128, 64, 64, 128, 128),
    'Wide (512-256-512)': (512, 256, 512)
}

results = []

print("Experimenting with different network architectures...")
print("="*60)

for name, layers in architectures.items():
    print(f"\nTraining: {name}...")
    
    # Create and train model
    mlp = MLPClassifier(
        hidden_layer_sizes=layers,
        activation='relu',
        solver='adam',
        max_iter=200,
        random_state=RANDOM_STATE,
        early_stopping=False,  # Disabled for multi-label compatibility
        verbose=False
    )
    
    clf = OneVsRestClassifier(mlp, n_jobs=-1)
    clf.fit(X_train, y_train)
    
    # Evaluate
    y_pred = clf.predict(X_test)
    
    results.append({
        'Architecture': name,
        'Layers': str(layers),
        'Accuracy': accuracy_score(y_test, y_pred),
        'Micro F1': f1_score(y_test, y_pred, average='micro', zero_division=0),
        'Macro F1': f1_score(y_test, y_pred, average='macro', zero_division=0)
    })
    
    print(f"  Accuracy: {results[-1]['Accuracy']:.4f}, Micro F1: {results[-1]['Micro F1']:.4f}")

# Display results
results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("ARCHITECTURE COMPARISON RESULTS")
print("="*80)
print(results_df.to_string(index=False))

---
## 6. Summary

### What was accomplished
1. Loaded preprocessed data from Lab 2 and vocabulary from Lab 4
2. Created binary feature vectors (Bag-of-Words encoding) for all samples
3. Trained a Neural Network with 128→64→128 hidden layers using MLPClassifier
4. Evaluated performance on test set
5. Compared Neural Network with Naive Bayes classifier
6. Experimented with different network architectures

### Key Findings
- Neural networks can capture non-linear relationships in text classification
- The MLPClassifier with ReLU activation and Adam optimizer provides good results
- For multi-label tasks, OneVsRestClassifier trains separate binary classifiers per class
- Network architecture affects performance, but larger isn't always better

In [ ]:
print("="*60)
print("LAB 5 SUMMARY")
print("="*60)
print(f"Input vocabulary: {VOCABULARY_PATH}")
print(f"Training samples: {len(df_train):,}")
print(f"Test samples: {len(df_test):,}")
print(f"Feature vector size: {X_train.shape[1]}")
print(f"Number of classes: {len(TOPIC_CLASSES)}")
print(f"\nBest Neural Network Metrics:")
print(f"  Subset Accuracy: {nn_metrics['Subset Accuracy']:.4f}")
print(f"  Micro F1: {nn_metrics['Micro F1']:.4f}")
print(f"  Macro F1: {nn_metrics['Macro F1']:.4f}")
print("="*60)